In [ ]:
def network_information(network, image) :
    print("="*50)
    print(" Network : ", network)
    print( " Array Size = {} x {}".format(array[0], array[1]))
    print("-"*30)

    print(" NETWORK INFORMATION ")
    print("-"*30)
    for i in range(len(image)) :
        print(" CONV LAYER "+ str(i+1))
        print("    Image   Size = {} x {}".format(image[i], image[i]))
        print("    Kernel  Size = {} x {}".format(kernel[i], kernel[i]))
        if network == 'VGG13' :
          print("    Channel Size = {} x {}".format(channel[i], channel[i+1]))
        elif network == 'Resnet13' :
          print("    Channel Size = {} x {}".format(channel[i], channel[i]))
    print("="*50)

In [ ]:
import math
import seaborn as sns
import matplotlib.pyplot as plt
import random
import numpy as np

class visualization :

  def __init__(self):
    pass

  def lookup_table_rect(self, PW_h, PW_w, filter_size): # array에서 좌표
    """
    reference pixel coordinate for IRIS version
    same as input feature coordinate (toeplitz?)
    """
    n_window_w, n_window_h = PW_w - filter_size + 1, PW_h - filter_size + 1 # filter의 갯수, height/width 방향
    lookup_table = np.empty(shape = (n_window_w * n_window_h, filter_size * filter_size), dtype=int)
    # print(lookup_table.shape)
    # print('1')
    lookup_table.fill(0)
    count = 0
    for x in range(n_window_h):
        for y in range(n_window_w): # 0 1
            for w in range(filter_size):
                for h in range(filter_size): # 0 1 2
                    #print(count,filter_size*w+h)
                    lookup_table[count][filter_size * w + h] = PW_w * (x + w) + y + h

            count=count+1
    # print(lookup_table)
    # print('2')
    return lookup_table

  def visualize(self, Array, Size):
    custom_color_map = []
    for i in range(Size):
        color = "#" + "%06x" % random.randint(0, 0xFFFFFF) # random palette
        custom_color_map.append(color)
    max_size = 15 #*(1 + int(max(Array.shape[1], Array.shape[0])/50)) # 15 * bigger size between col and row, increases 1 per 50
    figr, axs = plt.subplots(figsize=(max_size, max_size))    #(15,15)

    cbar_kws = {
                "orientation":"vertical",
                "drawedges":True,
                } # color bar keyword arguments
    axs = sns.heatmap(Array, annot=True, linewidth = 1, cbar_kws=cbar_kws, cmap=custom_color_map, fmt='g' )

  def generate_array_IRIS(self, kernel, n_window_w, n_window_h, IC_tiled, OC_tiled, array_row, array_col, method):
        K = int(kernel)
        n_window_w, n_window_h = int(n_window_w), int(n_window_h)
        if method == 'im2col' :
            C, M = int(IC_tiled) + 1, int(OC_tiled)
        else :
            C, M = int(IC_tiled), int(OC_tiled)
        PW_w,PW_h = K + n_window_w -1, K + n_window_h - 1

        total_col = M * n_window_w * n_window_h
        total_row = PW_h * PW_w * C # default row size

        array_row, array_col = int(array_row), int(array_col)

        print("SRAM SIZE : (Row: {}, Column: {})".format(array_row, array_col))
        print("TILED SIZE : (Row: {}, Column: {})".format(total_row, total_col))
        print("-"*40)
        print("="*40)

        Array = np.empty(shape=(1024,1024)) # the maximum size with scale down (8x) is 1024x1024.
        Array.fill(np.nan)
        #tcol = int(col/M) # small block for single channel
        tcol = n_window_w * n_window_h
        trow = int(total_row/C) # small block for single channel
        ###########################################


        #print(reference)
        #print(Array)
        # if overlap_h == overlap_w:
        reference = self.lookup_table_rect(PW_h, PW_w, K)
        count = 0
        for i in range(C): # number of Channel
            for j in range(0,total_col,M): # Filters
                for k in range(trow): # row of small block (single channel)
                    for l in range(M): # Tiled filters (diverse filters)
                        #print(int(j/M),count,reference[int(j/M)][count]+H*H*i,j+l)
                        '''
                        horizontal: filters
                        vertical: lookup table coordinate + input order (ex. 1245, 2356, ...)
                        value: filter number (different channel, filter number)
                        reference[int(j/M)][count]:
                        '''
                        Array[reference[int(j/M)][count]+PW_h*PW_w*i][j+l] = 1 + i + C*l
                        #int(reference[i][count])
                    count = count+1
                    if count > K*K-1: #coordinate of filter pixel
                        count = 0

        Array = Array[:array_row, :array_col]
        # else:
        #     for i in range(M): # filters
        #         for j in range(C): # channels
        #             for k in range(tcol): #row of small block
        #                 for l in range(K*K): #
        #                     Array[j*PW_h*PW_w+ K*k+l][tcol*i+k] = 1+l
        #                     #print("x:{}, y:{} = {}",tcol*i+k,j*SDK_h*SDK_w+ K*k+l,1+l)
        self.visualize(Array,C*M)

def im2col (image_col, image_row, filter_col, filter_row, in_channel, out_channel, array_row, array_col) :

    col_slide = image_col - filter_col + 1
    row_slide = image_row - filter_row + 1

    col_cycle = math.ceil(out_channel/array_col)

    # 우선 첫 cycle의 매핑을 보여주기위해 세팅
    if col_cycle > 1 :
        o_ct = array_col
    else :
        o_ct = out_channel

    i_ct = math.floor(array_row/(filter_col*filter_row))
    row_cycle = math.ceil(in_channel/i_ct)
    total_cycle = col_slide * row_slide * row_cycle * col_cycle

    return total_cycle, i_ct, o_ct

def SDK (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col) :

    row_vector = filter_row * filter_col * in_channel
    col_vector = out_channel

    used_row = math.ceil(row_vector/array_row)
    used_col = math.ceil(col_vector/array_col)

    new_array_row = array_row * used_row
    new_array_col = array_col * used_col

    # initialize
    cycle = []
    w = [] # pw 크기
    w.append(filter_row*filter_col)
    cycle.append(used_row*used_col*(image_row-filter_row+1)*(image_col-filter_col+1))

    i=0
    while True :
        i += 1
        pw_row = filter_row + i - 1
        pw_col = filter_col + i - 1
        pw = pw_row * pw_col
        if pw*in_channel <= new_array_row and i * i * out_channel <= new_array_col :
            parallel_window_row = math.ceil((image_row - (filter_row + i) + 1)/i) + 1
            parallel_window_col = math.ceil((image_col - (filter_col + i) + 1)/i) + 1

            if parallel_window_row * parallel_window_row * used_row * used_col <= cycle[0] :
                del cycle[0]
                del w[0]
                cycle.append(parallel_window_row * parallel_window_col * used_row * used_col)
                w.append(pw)

        else :
            break

    return cycle, w

# ceil : up, floor : down
def vw_sdk (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col) :

    i = 0 # initialize # overlap col
    j = 1 # overlap row

    reg_total_cycle = [] # initialize
    reg_overlap_row = []
    reg_overlap_col = []
    reg_row_cycle = []
    reg_col_cycle = []
    reg_ICt = []
    reg_OCt = []

    while True :
        try :
            i += 1
            if (i + filter_col) > image_col :
                i = 1
                j += 1
                if j + filter_row > image_row :
                    break

            # for parallel_window computing
            reg_N_parallel_window_row = math.ceil((image_row - (filter_row + i) + 1)/i) + 1
            reg_N_parallel_window_col = math.ceil((image_col - (filter_col + j) + 1)/j) + 1

            # for cycle computing
            # Tiled IC
            if in_channel == 3 :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                if ICt > in_channel :
                    ICt = 3
                row_cycle = math.ceil(in_channel / ICt)
            else :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                row_cycle = math.ceil(in_channel / ICt)

            # Tiled OC
            OCt =  math.floor(array_col / (i * j))
            col_cycle = math.ceil(out_channel / OCt)

            reg_N_of_computing_cycle = reg_N_parallel_window_row * reg_N_parallel_window_col \
                                    * row_cycle * col_cycle

            if i == 1 : # initialize
                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)

            if reg_total_cycle[0] > reg_N_of_computing_cycle :
                del reg_total_cycle[0]
                del reg_overlap_row[0]
                del reg_overlap_col[0]
                del reg_row_cycle[0]
                del reg_col_cycle[0]
                del reg_ICt[0]
                del reg_OCt[0]

                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)


        except ZeroDivisionError :
            continue

    return reg_total_cycle[0], reg_overlap_col[0], reg_overlap_row[0], reg_row_cycle[0], reg_col_cycle[0], reg_ICt[0], reg_OCt[0]

def result_vw (image, kernel, IC, OC, array_row, array_col, method) :

    VWSDK_height = []
    VWSDK_width = []
    AR_cycle = []
    AC_cycle = []
    VW_IC_tiled = []
    VW_OC_tiled = []

    CC=[]
    print("="*50)
    print(" RESULTS of COMPUTING CYCLES")
    print("-"*30)

    T_cycle_vw, VWSDK_h, VWSDK_w, ARC, ACC, tiled_IC, tiled_OC = vw_sdk(image, image, kernel, kernel, IC, OC, array_row, array_col)
    CC.append(T_cycle_vw)
    VWSDK_height.append(VWSDK_h)
    VWSDK_width.append(VWSDK_w)
    AR_cycle.append(ARC)
    AC_cycle.append(ACC)
    VW_IC_tiled.append(tiled_IC)
    VW_OC_tiled.append(tiled_OC)

    T_cycle_im, i_ct, o_ct = im2col(image, image, kernel, kernel, IC, OC, array_row, array_col)
    T_cycle_SDK, pw = SDK(image, image, kernel, kernel, IC, OC, array_row, array_col)

    SDK_w, SDK_h = math.sqrt(pw[0])-kernel+1, math.sqrt(pw[0])-kernel+1
    SDK_ict = math.floor(array_row /((kernel + SDK_w - 1)*(kernel + SDK_h - 1)))
    SDK_oct = math.floor(array_col / (SDK_w * SDK_h))

    print("    Im2col = {}".format(T_cycle_im))
    print("     S D K = {}".format(T_cycle_SDK[0]))
    print("     S D K = {}".format(pw[0]))
    print("    VW-SDK = {}".format(CC[0]))
    print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel + VWSDK_width[0]-1, kernel + VWSDK_height[0]-1, VW_IC_tiled[0], VW_OC_tiled[0]))
    print("="*50)


In [ ]:
# definations

# ckecking marginal space
def marginal_optimizable_row(image, kernel, pw_row):
  if (image-pw_row)%(pw_row - kernel +1) != 0:
    return True
  else:
    return False

def marginal_optimizable_col(image, kernel, pw_col):
  if (image-pw_col)%(pw_col - kernel +1) != 0:
    return True
  else:
    return False

def N_parallel_window(image,kernel,pw_row,pw_col):
  return (math.ceil((image-pw_row) /(pw_row - kernel +1)) + 1)* (math.ceil((image-pw_col) /(pw_col - kernel +1)) + 1)

def N_parallel_window_so(image,kernel,pw_row,pw_col):
  return (math.floor((image-pw_row) /(pw_row - kernel +1)) + 1)* (math.floor((image-pw_col) /(pw_col - kernel +1)) + 1)

In [ ]:
# cc considering marginal space, and depth

# cc considering marginal space
def cc_optimization_with_mo(image, kernel, ic, oc, ar, ac, pw_row, pw_col, pw_ic, pw_oc):
  No_conv = (pw_row - kernel + 1) * (pw_col - kernel + 1)
  print("No. of conv in one original VWSDK PW is", No_conv)

  No_parallel_window = N_parallel_window(image,kernel,pw_row,pw_col)
  print("No. of PW for the original VWSDK (one tile) is", No_parallel_window)

  optimal_N_parallel_window = No_parallel_window
  so_row = 0 #square-optimized
  so_col = 0
  No_cells = pw_row * pw_col #CIM array row occupied
  No_cells_so = No_cells
  ICt = pw_ic

  moX_row = 0 #marginal_space_window_x_direction
  moX_col = 0

  moY_row = 0
  moY_col = 0

  marginal_space_row = 0
  marginal_space_col = 0

  No_of_moX = 0
  No_of_moY = 0
  found_so = False
  ICmX = 0
  ICmY = 0

  No_windows = 0
  No_windows_so = 0
  No_remaining_parallel_window = No_parallel_window

  do_row = 0 #depth-wise optimized
  do_col = 0
  No_cells_do = No_cells
  mICt = 0

  mmoX_row = 0 #depth-wise marginal
  mmoX_col = 0

  mmoY_row = 0
  mmoY_col = 0

  mmarginal_space_row = 0
  mmarginal_space_col = 0

  No_of_mmoX = 0
  No_of_mmoY = 0

  mICmX = 0
  mICmY = 0


  for i in range(1, int(pow(No_conv, 1 / 2))+1):
        if No_conv % i == 0 and found_so == False:
          print("FACTOR: " + str(i) +"*"+str(int(No_conv / i)))
          sw_row = i + kernel -1
          sw_col = int(No_conv / i) + kernel -1
          # print("sw_pw is", sw_row, "*", sw_col)
          # print("cc is", N_parallel_window(image,kernel,sw_row,sw_col))
          # print(N_parallel_window(image,kernel,sw_row,sw_col))
          # print(ptimal_N_parallel_window)
          if (N_parallel_window(image,kernel,sw_row,sw_col)<= optimal_N_parallel_window) and (sw_row*sw_col < No_cells) :
            optimal_N_parallel_window = N_parallel_window(image,kernel,sw_row,sw_col)
            so_row = sw_row
            so_col = sw_col
            No_cells_so = sw_row*sw_col
            ICt = ar // No_cells_so
            found_so = True
            print("!!!!!found_so", found_so)
            print(so_row, so_col, "square-inclined window is better than the original PW", pw_row, pw_col)
            if marginal_optimizable_row(image, kernel, so_row):
              marginal_space_row = (image-so_row) % (so_row - kernel +1)
              print("marginal_space_row is",marginal_space_row) # 4
              moX_row = marginal_space_row + kernel - 1
              moX_col = (pw_row * pw_col) // moX_row
              ICmX = ar // (moX_row * moX_col)
              No_of_moX = math.ceil(((image-moX_col)/(moX_col - kernel +1)))+1
              print("marginal_window_size on rows has size", moX_row, moX_col, ", and No_of_moX is", No_of_moX)
            else:
              print("There is no marginal rows to be optimzed.")
              No_of_moX=0

            if marginal_optimizable_col(image, kernel, so_col):
              marginal_space_col = (image-so_col) % (so_col - kernel +1)
              print("marginal_space_col is",marginal_space_col) # 4
              moY_col = marginal_space_col + kernel - 1
              moY_row = (pw_row * pw_col) // moY_col
              ICmY = ar // (moY_row * moY_col)
              No_of_moY = math.ceil(((image-moY_row)/(moY_row - kernel +1)))+1
              print("marginal_window_size on cols has size", moY_row, moY_col, ", and No_of_moY is", No_of_moY)
            else:
              print("There is no marginal cols to be optimzed.")
              No_of_moY=0

            No_windows_so = N_parallel_window_so(image,kernel,so_row,so_col)
            No_windows = ( No_windows_so + No_of_moX + No_of_moY )
            # print("!!!!!N_parallel_window_so is!!!!", No_windows_so)
            print("!!!!!No_windows with so_mo is!!!!", No_windows)


          else:
            print("This pair is not optimized")
            No_windows = N_parallel_window(image,kernel,sw_row,sw_col)

  if found_so == False:
            print("found_so", found_so)
            if marginal_optimizable_row(image, kernel, pw_row):
              marginal_space_row = (image-pw_row) % (pw_row - kernel +1)
              print("marginal_space_row is",marginal_space_row) # 4
              moX_row = marginal_space_row + kernel - 1
              moX_col = (ar/pw_ic) // moX_row
              ICmX = ar // (moX_row * moX_col)
              No_of_moX = math.ceil(((image-moX_col)/(moX_col - kernel +1)))+1
              print("marginal_window_size on rows", moX_row, moX_col, "No_of_moX is", No_of_moX)
            else:
              print("There is no marginal rows to be optimzed.")
              No_of_moX=0

            if marginal_optimizable_col(image, kernel, pw_col):
              marginal_space_col = (image-pw_col) % (pw_col - kernel +1)
              print("marginal_space_col is",marginal_space_col) # 4
              moY_col = marginal_space_col + kernel - 1
              moY_row = (ar/pw_ic) // moY_col
              ICmY = ar // (moY_row * moY_col)
              No_of_moY = math.ceil(((image-moY_row)/(moY_row - kernel +1)))+1
              print("marginal_window_size on cols", moY_row, moY_col, "No_of_moY is", No_of_moY)
            else:
              print("There is no marginal cols to be optimzed.")
              No_of_moY=0

            No_windows_so = N_parallel_window_so(image,kernel,pw_row,pw_col)
            No_windows = ( No_windows_so + No_of_moX + No_of_moY )
            print("!!!!!N_parallel_window_so is!!!!", No_windows_so)
            print("!!!!!No_windows with so_mo is!!!!", No_windows)


# calculate variable depeth
  print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
  remaining_channels = ic % ICt
  reg_remaining_channels = remaining_channels
  print("remaining channels", remaining_channels)
  No_windows = No_windows * (ic // ICt)
  print("!!!!!No_windows with so_mo is!!!!", No_windows)
  max_conv = ac // oc #16
  print("max_conv", max_conv)
  found_max = False
  found_conv = False
  # found_so = False
          # No_remaining_parallel_window

  prune = 0

  while found_max==False and (remaining_channels >0):
    # try:
      print("PRUNING", prune, "CHANNELS")
      remaining_channels -= prune
      print("remaining channels", remaining_channels)
      for i in range(int(pow(max_conv, 1 / 2))+1,1, -1):
        if (max_conv % i == 0) and (int(max_conv / i) <= image):
                  print("FACTOR: " + str(i) +"*"+str(int(max_conv / i)))
                  dw_row = i + kernel -1
                  dw_col = int(max_conv / i) + kernel -1
                  print("dw_pw is", dw_row, "*", dw_col)
                  print("cc is", N_parallel_window(image,kernel,dw_row,dw_col))
                  if marginal_optimizable_row(image, kernel, dw_row):
                          print(image,kernel,dw_row)
                          mmarginal_space_row = (image-dw_row) % (dw_row - kernel +1)
                          print("marginal_space_row is",mmarginal_space_row) # 2
                          mmoX_row = mmarginal_space_row + kernel - 1 #6
                          mmoX_col = (ar / pw_ic) // mmoX_row
                          mICmX = ar // (mmoX_row * mmoX_col)
                          No_of_mmoX = math.ceil(((image-mmoX_col)/(mmoX_col - kernel +1)))+1
                          print("marginal_window_size on rows", mmoX_row, mmoX_col, mICmX, "No_of_moX", No_of_mmoX)
                  else:
                          print("There is no marginal rows to be optimzed.")
                          No_of_mmoX =0

                  if marginal_optimizable_col(image, kernel, dw_col):
                          mmarginal_space_col = (image-dw_col) % (dw_col - kernel +1)
                          print("marginal_space_col is",mmarginal_space_col) # 1
                          mmoY_col = mmarginal_space_col + kernel - 1 #3
                          mmoY_row = (pw_row * pw_col) // mmoY_col #7
                          mICmY = ar // (mmoY_row * mmoY_col)
                          No_of_mmoY = math.ceil(((image-mmoY_row)/(mmoY_row - kernel +1)))+1
                          print("marginal_window_size on cols", mmoY_row, mmoY_col, mICmY, "No_of_moY", No_of_mmoY)
                  else:
                          print("There is no marginal cols to be optimzed.")
                          No_of_mmoY=0

            #               No_windows_so = N_parallel_window_so(image,kernel,so_row,so_col)
            # No_windows = ( No_windows_so + No_of_moX + No_of_moY )
                  No_window_with_marginal = N_parallel_window_so(image,kernel,dw_row,dw_col) + No_of_mmoX + No_of_mmoY
                  print("No_window_with_marginal",No_window_with_marginal)

                  # No_windows += optimal_N_parallel_window + No_of_mmoX + No_of_mmoY
                  if (No_window_with_marginal<= No_remaining_parallel_window) and (dw_row*dw_col * (remaining_channels) <= ar) :
                      found_max = True
                      optimal_N_parallel_window = N_parallel_window(image,kernel,dw_row,dw_col)
                      do_row = dw_row
                      do_col = dw_col
                      No_cells_do = dw_row*dw_col
                      mICt = ar // No_cells_do
                      # N_parallel_window_so(image,kernel,so_row,so_col)
                      print(do_row, do_col, "is better than the original PW", pw_row, pw_col)
                      # print(optimal_N_parallel_window, "optimal_N_parallel_window")
                      No_windows += No_window_with_marginal

                      # if marginal_optimizable_row(image, kernel, do_row):
                      #     mmarginal_space_row = (image-do_row) % (do_row - kernel +1)
                      #     print("marginal_space_row is",marginal_space_row) # 4
                      #     mmoX_row = marginal_space_row + kernel - 1
                      #     mmoX_col = (pw_row * pw_col) // mmoX_row

                      #     mICmX = ar // (moX_row * mmoX_col)

                      #     No_of_mmoX = math.ceil(((image-mmoX_col)/(mmoX_col - kernel +1)))+1

                      #     print("marginal_window_size on rows", mmoX_row, mmoX_col, mICmX, "No_of_moX", No_of_mmoX)
                      # else:
                      #     print("There is no marginal rows to be optimzed.")

                      # if marginal_optimizable_col(image, kernel, do_col):
                      #     mmarginal_space_col = (image-do_col) % (do_col - kernel +1)
                      #     print("marginal_space_col is",marginal_space_col) # 4
                      #     mmoY_col = marginal_space_col + kernel - 1
                      #     mmoY_row = (pw_row * pw_col) // mmoY_col

                      #     mICmY = ar // (mmoY_row * mmoY_col)

                      #     No_of_mmoY = math.ceil(((image-moY_row)/(mmoY_row - kernel +1)))+1

                      #     print("marginal_window_size on cols", mmoY_row, mmoY_col, mICmY, "No_of_moY", No_of_mmoY)
                      # else:
                      #     print("There is no marginal cols to be optimzed.")
                      # No_windows += optimal_N_parallel_window + No_of_mmoX + No_of_mmoY
                      print("OVERALL Computing Cycle: ", No_windows)
                      break
                  else:
                      print("This pair is not allowed")

      prune =1




  return No_windows

In [ ]:
#%%
import math
# from function_1 import *
import matplotlib.pyplot as plt
# import argparse

'''
Mapping Method
im2col, SDK, VW-SDK
The input/output channel have to be scaled-down (8x)
ex) 3x3x128x256 -> 3x3x16x32


28	5	16x32
28	5	32x96
14	5	16x48
14	5	24x64
14	5	24x64
14	5	32x64
14	5	32x128
7	5	32x128
7	5	28x128
'''


# image, kernel, ic, oc, ar, ac, method = 14, 5, 32, 128, 512, 512, 'VW-SDK'
image, kernel, ic, oc, ar, ac, method = 5, 3, 5, 2, 40, 3, 'VW-SDK'

group = 1
ic = ic/group
oc = oc/group

print("="*50)
print("INFORMATION")
print("-"*30)
print("    Array   Size = {} x {}".format(ar, ac))
print("    Image   Size = {} x {}".format(image, image))
print("    Kernel  Size = {} x {}".format(kernel, kernel))
print("    Channel Size = {} x {}".format(ic, oc))
print("-"*30)

result_vw(image, kernel, ic, oc, ar, ac, method)

INFORMATION
------------------------------
    Array   Size = 40 x 3
    Image   Size = 5 x 5
    Kernel  Size = 3 x 3
    Channel Size = 5.0 x 2.0
------------------------------
 RESULTS of COMPUTING CYCLES
------------------------------
    Im2col = 18
     S D K = 18
     S D K = 9
    VW-SDK = 18
      - Optimal shape of PW = 3 x 3 x 4 x 3


In [ ]:
# image, kernel, ic, oc, ar, ac = 14, 5, 16, 48, 512, 256
pw_row, pw_col, pw_ic, pw_oc = 9, 7, 8, 32
print("RESULT CC")
print(ic)
cc_optimization_with_mo(image, kernel, ic, oc, ar, ac, pw_row, pw_col, pw_ic, pw_oc)

RESULT CC
8.0
No. of conv in one original VWSDK PW is 15
No. of PW for the original VWSDK (one tile) is 8
FACTOR: 1*15
This pair is not optimized
FACTOR: 3*5
This pair is not optimized
found_so False
There is no marginal rows to be optimzed.
marginal_space_col is 1
marginal_window_size on cols 12.0 5 No_of_moY is 2
!!!!!N_parallel_window_so is!!!! 6
!!!!!No_windows with so_mo is!!!! 8
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
remaining channels 0.0
!!!!!No_windows with so_mo is!!!! 8.0
max_conv 16.0


8.0

In [ ]:
# new codes, given parameters, group number and vwsdk shape, return cc

def vw_sdk (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col) :

    i = 0 # initialize # overlap col
    j = 0 # overlap row

    reg_total_cycle = [] # initialize
    reg_overlap_row = []
    reg_overlap_col = []
    reg_row_cycle = []
    reg_col_cycle = []
    reg_ICt = []
    reg_OCt = []

    while True :
        try :
            i += 1
            if (i + filter_col) > image_col :
                i = 1
                j += 1
                if j + filter_row > image_row :
                    break

            # for parallel_window computing
            reg_N_parallel_window_row = math.ceil((image_row - (filter_row + i) + 1)/i) + 1
            reg_N_parallel_window_col = math.ceil((image_col - (filter_col + j) + 1)/j) + 1

            # for cycle computing
            # Tiled IC
            if in_channel == 3 :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                if ICt > in_channel :
                    ICt = 3
                row_cycle = math.ceil(in_channel / ICt)
            else :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                row_cycle = math.ceil(in_channel / ICt)

            # Tiled OC
            OCt =  math.floor(array_col / (i * j))
            col_cycle = math.ceil(out_channel / OCt)

            reg_N_of_computing_cycle = reg_N_parallel_window_row * reg_N_parallel_window_col \
                                    * row_cycle * col_cycle

            if i == 1 : # initialize
                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)

            if reg_total_cycle[0] > reg_N_of_computing_cycle :
                del reg_total_cycle[0]
                del reg_overlap_row[0]
                del reg_overlap_col[0]
                del reg_row_cycle[0]
                del reg_col_cycle[0]
                del reg_ICt[0]
                del reg_OCt[0]

                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)


        except ZeroDivisionError :
            continue

    return reg_total_cycle[0], reg_overlap_col[0], reg_overlap_row[0], reg_row_cycle[0], reg_col_cycle[0], reg_ICt[0], reg_OCt[0]


def result_vw (image, kernel, IC, OC, array_row, array_col, method) :

    VWSDK_height = []
    VWSDK_width = []
    AR_cycle = []
    AC_cycle = []
    VW_IC_tiled = []
    VW_OC_tiled = []

    CC=[]
    print("="*50)
    print(" RESULTS of COMPUTING CYCLES")
    print("-"*30)

    T_cycle_vw, VWSDK_h, VWSDK_w, ARC, ACC, tiled_IC, tiled_OC = vw_sdk(image, image, kernel, kernel, IC, OC, array_row, array_col)
    CC.append(T_cycle_vw)
    VWSDK_height.append(VWSDK_h)
    VWSDK_width.append(VWSDK_w)
    AR_cycle.append(ARC)
    AC_cycle.append(ACC)
    VW_IC_tiled.append(tiled_IC)
    VW_OC_tiled.append(tiled_OC)

    T_cycle_im, i_ct, o_ct = im2col(image, image, kernel, kernel, IC, OC, array_row, array_col)
    T_cycle_SDK, pw = SDK(image, image, kernel, kernel, IC, OC, array_row, array_col)

    SDK_w, SDK_h = math.sqrt(pw[0])-kernel+1, math.sqrt(pw[0])-kernel+1
    SDK_ict = math.floor(array_row /((kernel + SDK_w - 1)*(kernel + SDK_h - 1)))
    SDK_oct = math.floor(array_col / (SDK_w * SDK_h))

    print("    Im2col = {}".format(T_cycle_im))
    print("     S D K = {}".format(T_cycle_SDK[0]))
    print("     S D K = {}".format(pw[0]))
    print("    VW-SDK = {}".format(CC[0]))
    print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel + VWSDK_width[0]-1, kernel + VWSDK_height[0]-1, VW_IC_tiled[0], VW_OC_tiled[0]))
    print("="*50)

In [ ]:
image, kernel, ic, oc, ar, ac = 7, 5, 32, 128, 512, 512

# image_col, image_row, filter_col, filter_row, in_channel, out_channel, array_row, array_col

vw_sdk (image, image, kernel, kernel, ic, oc, ar, ac)

(12, 1, 2, 2, 1, 17, 256)

In [ ]:
def window (image, kernel, in_channel, out_channel, array_row, array_col):
  pw_row, pw_col, ict, oct, cc = 0,0,0,0,0

  b_pw_row, b_pw_col = kernel,kernel
  b_ict = math.floor(array_row /((b_pw_row)*(b_pw_col)))
  b_oct = math.floor(array_col / ((b_pw_row-kernel+1) * (b_pw_col-kernel+1)))
  b_row_cycle = math.ceil(in_channel / b_ict)
  b_col_cycle = math.ceil(out_channel / b_oct)
  b_reg_N_parallel_window = N_parallel_window(image,kernel,b_pw_row,b_pw_col)
  b_reg_N_of_computing_cycle = b_reg_N_parallel_window * b_row_cycle * b_col_cycle



  f_pw_row, f_pw_col, f_ict, f_oct, f_cc = 0,0,0,0,0



  for pw_row in range (kernel,image):
    for pw_col in range(kernel,image):
      ict = math.floor(array_row /((pw_row)*(pw_col)))
      oct = math.floor(array_col / ((pw_row-kernel+1) * (pw_col-kernel+1)))
      row_cycle = math.ceil(in_channel / ict)
      col_cycle = math.ceil(out_channel / oct)
      reg_N_parallel_window = N_parallel_window(image,kernel,pw_row,pw_col)
      reg_N_of_computing_cycle = reg_N_parallel_window * row_cycle * col_cycle

      if reg_N_of_computing_cycle < b_reg_N_of_computing_cycle:
        b_reg_N_of_computing_cycle = reg_N_of_computing_cycle
        f_pw_row, f_pw_col, f_ict, f_oct, f_cc = pw_row, pw_col, ict, oct, reg_N_of_computing_cycle

  return f_pw_row, f_pw_col, f_ict, f_oct, f_cc

window(image, kernel, ic, oc, ar, ac)

(5, 6, 17, 256, 12)

In [ ]:
def N_parallel_window(image,kernel,pw_row,pw_col):
  return (math.ceil((image-pw_row) /(pw_row - kernel +1)) + 1)* (math.ceil((image-pw_col) /(pw_col - kernel +1)) + 1)

def N_parallel_window_so(image,kernel,pw_row,pw_col):
  return (math.floor((image-pw_row) /(pw_row - kernel +1)) + 1)* (math.floor((image-pw_col) /(pw_col - kernel +1)) + 1)

